<a href="https://colab.research.google.com/github/ipavlopoulos/paremia/blob/main/unlocated_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!gdown 1wHp7SyB_tfWKVayzBhhYla3Ue2JcynuG
!gdown 1O9t4DjeZAf14mev2IjfUtMtBC1Gts_Lw
!git clone https://github.com/ipavlopoulos/paremia.git
!pip install transformers

In [2]:
import os
import pandas as pd
corpus_path = "data/undated.csv"
if not os.path.exists(corpus_path):
  corpus_path = 'https://raw.githubusercontent.com/ipavlopoulos/paremia/main/data/unlocated.csv'
unlocated = pd.read_csv(corpus_path, index_col=0)

In [6]:
from paremia.lib.classification import *
model = GrBertC()
model.load_state_dict(torch.load("bert-gr-c.pt"))

<All keys matched successfully>

In [ ]:
from tqdm.notebook import tqdm 
import numpy as np

def predict(model, dataloader, device="cpu"):
    predictions = []
    model.eval()
    with torch.no_grad():
        for batch_id, (val_input, val_label) in tqdm(enumerate(dataloader)):
            val_label = val_label.to(device)
            mask = val_input['attention_mask'].to(device)
            input_id = val_input['input_ids'].squeeze(1).to(device)
            output = model(input_id, mask)
            gold = np.argmax(val_label.cpu().detach().numpy(), axis=1)
            pred = np.argmax(output.cpu().detach().numpy(), axis=1)
            predictions.extend(pred)
    return predictions

unlocated['area'] = 'Ήπειρος' 
loader = torch.utils.data.DataLoader(Dataset(unlocated, max_length=32), batch_size=32, drop_last=False)
predictions = predict(model.to("cpu"), loader, "cpu")

0it [00:00, ?it/s]

In [18]:
predicted_locs = [idx2loc[p] for p in predictions]
unlocated['area'] = predicted_locs

In [20]:
unlocated.to_csv('unlocated_classified.csv')